# Courtright PBR Extraction

Courtright Reservior: https://www.google.com/maps/place/Courtright+Reservoir/@37.1016292,-119.0217487,13z/data=!4m6!3m5!1s0x809596e61f5a4355:0xf1aa6387a6022d1e!8m2!3d37.1019078!4d-118.9732411!16zL20vMGQ3cjhn?entry=ttu 

## 1. Organize data
Create a folder under `semantic_SfM/data` and organize your data following the structures below. 

Agisoft:
```
semantic_SfM/data
    ├── courtright
        ├── DJI_photos
        │       ├── DJI_0000.JPG
        │       ├── DJI_0001.JPG
        │       ├── ...
        │       └── DJI_0100.JPG
        ├── SfM_products
        │       ├── agisoft_cameras.xml
        │       ├── model.jpg
        │       ├── model.mtl
        │       ├── model.obj
        │       └── agisoft_model.las   
        ├── segmentations
        └── associations

```

## 2. Create 2D Segmentation using SAM

In [1]:
from ssfm.image_segmentation import ImageSegmentation
import os

In [4]:
sam_params = {}
sam_params['model_name'] = 'sam'
sam_params['model_path'] = '../semantic_SfM/sam/sam_vit_h_4b8939.pth'
sam_params['model_type'] = 'vit_h'
sam_params['device'] = 'cuda:3'
sam_params['points_per_side'] = 32
sam_params['pred_iou_thresh'] = 0.96
sam_params['stability_score_thresh'] = 0.96
sam_params['crop_n_layers'] = 2

In [5]:
image_segmentor = ImageSegmentation(sam_params)   
image_segmentor.set_distortion_correction('../data/courtright/SfM_products/agisoft_cameras.xml')
image_folder_path = '../data/courtright/DJI_photos'
segmentation_folder_path = '../data/courtright/segmentations'
image_paths = [os.path.join(image_folder_path, f) for f in os.listdir(image_folder_path) if f.endswith('.JPG')]
image_paths.sort()
# keep the part of the images
image_paths = image_paths[200:]
image_segmentor.batch_predict(image_paths, segmentation_folder_path, maximum_size=10000, save_overlap=True)

Processing image 1/1.


In [1]:
# simple segmentation filter
from ssfm.mask_filter import SimpleMaskFilter

In [2]:
segmentation_folder_path = "../data/courtright/segmentations"

configs = {
    'window_size': 5,
    'depth_folder': "../data/courtright/associations/depth",
    'output_folder': "../data/courtright/segmentations_filtered",
    'area_upper_threshold': 6,
    'area_lower_threshold': 0.01,
    'erosion_kernel_size': 5,
    'erosion_iteration':1,
    'camera_parameter_file': "../data/courtright/SfM_products/agisoft_cameras.xml",
    'background_mask': True
}


mask_filter = SimpleMaskFilter(configs)

#mask_filter.filter_segmentation_file('../../data/courtright/segmentations/DJI_0650.npy')
mask_filter.filter_batch_processes(segmentation_folder_path, num_processes=16)

100%|██████████| 294/294 [15:44<00:00,  3.21s/it]


## 3. Create projection associations

In [3]:
from ssfm.probabilistic_projection import *
import time

In [4]:
pointcloud_projector = PointcloudProjection(depth_filtering_threshold=0.08)
pointcloud_projector.read_camera_parameters('../data/courtright/SfM_products/agisoft_cameras.xml')
pointcloud_projector.read_pointcloud('../data/courtright/SfM_products/agisoft_model.las')
pointcloud_projector.read_mesh('../data/courtright/SfM_products/agisoft_model.obj')

In [5]:
# batch project
image_folder_path = '../data/courtright/DJI_photos'
save_folder_path = '../data/courtright/associations'

image_list = [f for f in os.listdir(image_folder_path) if f.endswith('.JPG')]
#pointcloud_projector.parallel_batch_project(image_list, save_folder_path)
pointcloud_projector.parallel_batch_project_joblib(image_list, save_folder_path, num_workers=16, save_depth=True)

Processing frames:  11%|█         | 32/294 [00:31<04:16,  1.02it/s]/home/zchen256/.local/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
Processing frames: 100%|██████████| 294/294 [13:00<00:00,  2.66s/it]


In [8]:
# build keyimage associations
from ssfm.keyimage_associations_builder import *

In [9]:
smc_solver = KeyimageAssociationsBuilder('../data/courtright/associations', '../data/courtright/segmentations_filtered')
smc_solver.build_associations()

100%|██████████| 294/294 [01:45<00:00,  2.78it/s]


In [9]:
smc_solver.find_min_cover()

| Metric                                                       | Count      | Percentage           |
----------------------------------------------------------------------------------------------------
| Number of points not covered by any image                    | 37250341   | 44.94                |
| Number of points covered by less than or equal to 1 image    | 48947394   | 59.05                |
| Number of points covered by less than or equal to 3 images   | 59865729   | 72.22                |
| Number of points covered by less than or equal to 5 images   | 65670931   | 79.23                |


In [8]:
# refine keyimage associations (optional)
smc_solver.refine(0.5)  # keep 0.5 of the original keyimage associations

100%|██████████| 294/294 [05:04<00:00,  1.04s/it]


## 4. Estimate memory usage

In [10]:
from ssfm.memory_calculator import memory_calculator

In [11]:
# pointcloud file
las_file = "../data/courtright/SfM_products/agisoft_model.las"
# image file sample
image_file = "../data/courtright/DJI_photos/DJI_0576.JPG"
# number of images
num_images = 294
# number of segmentation ids for each point in the point cloud
num_segmentation_ids = 5

memory_calculator(las_file, image_file, num_images, num_segmentation_ids)

+----------------------------------------+----------------------+
|              Memory Type               | Memory Required (GB) |
+----------------------------------------+----------------------+
|      Segmentation for each image       | 0.037181854248046875 |
| Pixel2point association for each image | 0.07436370849609375  |
| Point2pixel association for each image | 0.21190671995282173  |
|                                        |                      |
|      Segmentation for all images       |  10.931465148925781  |
| Pixel2point association for all images |  21.862930297851562  |
| Point2pixel association for all images |  62.30057566612959   |
|          pc_segmentation_ids           |  1.0595335997641087  |
|         pc_segmentation_probs          |  1.0595335997641087  |
|          keyimage_association          |  15.575143916532397  |
|                 Total                  |  112.78918222896755  |
+----------------------------------------+----------------------+


## 5. Run object registration

In [12]:
from ssfm.object_registration import *

In [ ]:
# Set paths
pointcloud_path = '../data/courtright/SfM_products/agisoft_model.las'
segmentation_folder_path = '../data/courtright/segmentations_filtered'
image_folder_path = '../data/courtright/DJI_photos'
association_folder_path = '../data/courtright/associations'

keyimage_associations_file_name = 'refined_associations_keyimage.npy'
keyimage_yaml_name = 'refined_keyimages.yaml'

In [ ]:
# Create object registration
obr = ObjectRegistration(pointcloud_path, segmentation_folder_path, association_folder_path)

# use the refined keyimage associations
#obr = ObjectRegistration(pointcloud_path, segmentation_folder_path, association_folder_path, keyimage_associations_file_name=keyimage_associations_file_name, keyimage_yaml_name=keyimage_yaml_name)

# Run object registration
#obr.object_registration(iou_threshold=0.5, save_semantics=True)
obr.object_registration(iou_threshold=0.5)

# save semantic point cloud
obr.save_semantic_pointcloud('../data/courtright/semantic_model.las')


In [8]:
add_semantics_to_pointcloud_flag = True
if add_semantics_to_pointcloud_flag:
    image_id = 293
    semantics_folder_path = os.path.join(association_folder_path, 'semantics', 'semantics_{}.npy'.format(image_id))
    save_las_path = os.path.join(association_folder_path, 'semantics', 'semantics_{}.las'.format(image_id))
    add_semantics_to_pointcloud(pointcloud_path, semantics_folder_path, save_las_path) 


add_semantics_to_pointcloud_flag = True
if add_semantics_to_pointcloud_flag:
    image_id = 293
    semantics_folder_path = os.path.join(association_folder_path, 'semantics', 'semantics_{}.npy'.format(image_id))
    save_las_path = os.path.join(association_folder_path, 'semantics', 'semantics_{}_updated.las'.format(image_id))
    add_semantics_to_pointcloud(pointcloud_path, semantics_folder_path, save_las_path) 

maximum of semantics:  73
number of unique semantics:  56


In [7]:
from ssfm.post_processing import PostProcessing

In [5]:
semantic_pc_file_path = '../data/courtright/associations/semantics/semantics_293.las'
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.shuffle_semantic_ids()
save_las_path = '../data/courtright/associations/semantics/semantics_293_shuffled.las'
post_processing.save_semantic_pointcloud(save_las_path)


semantic_pc_file_path = '../data/courtright/associations/semantics/semantics_293_updated.las'
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.shuffle_semantic_ids()
save_las_path = '../data/courtright/associations/semantics/semantics_293_shuffled_updated.las'
post_processing.save_semantic_pointcloud(save_las_path)

Number of unique semantics:  869
